## Random Graphs

### Helper Functions

Randomly generate an adjancency matrix...

In [ ]:
import timeit
import random
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations, groupby

# Size of NxN Matrix
n = 8

# Moved to global scopee to allow for timing
bf_bit_string, bf_cost = None, None

def random_row():
    row = list(np.random.choice((0, 1, 2, 3, 4, 5),size=n))
    
    # Make sure there's no isolated nodes
    if sum(row) == 0:
        row = random_row()
    return row

def random_matrix():
    rows = []
    for i in range(0, n):
        rows.append(random_row())
    numpy_matrix = np.matrix(rows)
    
    return numpy_matrix

def random_graph():
    np_matrix = random_matrix()
    g = nx.convert_matrix.from_numpy_matrix(
        np_matrix, 
        parallel_edges=False, 
        create_using=nx.Graph
    )
    
    # nx.draw(g, node_color='lightblue', 
    #     with_labels=True, 
    #     node_size=500)
    
    return g, np_matrix

### Brute Force

In [ ]:
def brute_force(graph, w):
    global bf_bit_string
    global bf_cost
    
    bf_bit_string = None
    bf_cost = None
    
    best_cost_brute = 0
    for b in range(2**n):
        x = [int(t) for t in reversed(list(bin(b)[2:].zfill(n)))]
        cost = 0
        for i in range(n):
            for j in range(n):
                cost = cost + w[i,j]*x[i]*(1-x[j])
        if best_cost_brute < cost:
            best_cost_brute = cost
            xbest_brute = x
    bf_bit_string = xbest_brute
    bf_cost = best_cost_brute
    return xbest_brute, best_cost_brute

### Variatonal Quantum EignSolver

In [ ]:
from qiskit import Aer
from qiskit.tools.visualization import plot_histogram
from qiskit.circuit.library import TwoLocal
from qiskit_optimization.applications import Maxcut, Tsp
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import SPSA, COBYLA
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit_optimization.algorithms import MinimumEigenOptimizer

def vqe(w):
    # Define our Qiskit Maxcut Instance
    max_cut = Maxcut(w)
    qp = max_cut.to_quadratic_program()

    # Translate to Ising Hamiltonian
    qubitOp, offset = qp.to_ising()

    # Setup our simulator
    algorithm_globals.random_seed = 123
    seed = 10598
    backend = Aer.get_backend('aer_simulator_statevector')
    quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)

    # Construct VQE
    spsa = SPSA(maxiter=300)
    cobyla = COBYLA(maxiter=300)
    ry = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', reps=5, entanglement='linear')
    vqe = VQE(ry, optimizer=spsa, quantum_instance=quantum_instance)

    # Run VQE
    compute_min_eigenval_time = timeit.timeit(lambda: vqe.compute_minimum_eigenvalue(qubitOp), number=1)
    result = vqe.compute_minimum_eigenvalue(qubitOp)

    # print results
    bit_string = list(max_cut.sample_most_likely(result.eigenstate))
    bit_string = map(int, bit_string)
    # print('energy:', result.eigenvalue.real)
    # print('time:', result.optimizer_time)
    # print('max-cut objective:', result.eigenvalue.real + offset)
    # print('solution:', bit_string)
    # print('solution objective:', qp.objective.evaluate(bit_string))
    
    return bit_string, (result.eigenvalue.real + offset), result.optimizer_time, compute_min_eigenval_time

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## Tests

In [4]:
# Generate 1,000 graphs
success = 0
failure = 0

# TODO: Change the number of tests, ad hoc
num_tests: int = 100

for graph_id in range(num_tests):
    global bf_bit_string
    global bf_cost
    
    # Generate Random Graph
    graph, matrix = random_graph()
    
    # Computing the weight matrix from the random graph
    w = np.zeros([n,n])
    for i in range(n):
        for j in range(n):
            temp = graph.get_edge_data(i,j,default=0)
            if temp != 0:
                w[i,j] = temp['weight']
    
    print(f'\nTest Case {graph_id}: ')
    print('------------' + '-' * (graph_id * 2))
    print()
    print(w)
    
    # Brute Force Approach
    #
    # Time brute force algorithm
    bf_time: float = timeit.timeit(lambda: brute_force(graph, w), number=1)
    
    # Parse the bit sting answer
    bf_x = ''.join(str(e) for e in bf_bit_string)
    bf_x_inverse = ''.join('1' if x == '0' else '0' for x in bf_x)
    
    # Print results
    print('\n\tBrute Force')
    print(f'\t\tSolution = {bf_x} [{bf_x_inverse}].')
    print(f'\t\tMaxcut = {str(bf_cost)}.')
    print(f'\t\tTime: {bf_time} seconds.')
    
    # VQE Approach
    #
    #
    vqe_bit_string, vqe_cost, optimizer_time, compute_minimum_eigenvalue_time = vqe(w)
    
    # Parse the bit sting answer
    vqe_x = ''.join(str(e) for e in vqe_bit_string)
    vqe_x_inverse = ''.join('1' if x == '0' else '0' for x in vqe_x)
    
    # Print results
    print('\n\tVariation Quantum Eignsolver (VQE):')
    print(f'\t\tSolution = {vqe_x} [{vqe_x_inverse}].')
    print(f'\t\tMaxcut = {str(vqe_cost)}.')
    print(f'\t\tOptimizer Time: {str(optimizer_time)} seconds.')
    print(f'\t\tCompmute Minimum Eigenvalue Time: {str(optimizer_time)} seconds.')
    print()
    
    # Check validity
    if bf_x == vqe_x or bf_x == vqe_x_inverse or vqe_x == bf_x_inverse:
        success += 1
    else:
        failure += 1


Test Case 0: 
------------

[[4. 3. 3. 1. 2. 4. 4. 5.]
 [3. 1. 1. 4. 2. 3. 1. 5.]
 [3. 1. 0. 2. 3. 2. 1. 2.]
 [1. 4. 2. 1. 4. 4. 4. 4.]
 [2. 2. 3. 4. 5. 1. 4. 2.]
 [4. 3. 2. 4. 1. 3. 4. 4.]
 [4. 1. 1. 4. 4. 4. 2. 4.]
 [5. 5. 2. 4. 2. 4. 4. 2.]]

	Brute Force
		Solution = 10011100 [01100011].
		Maxcut = 53.0.
		Time: 0.009471814963035285 seconds.

	Variation Quantum Eignsolver (VQE):
		Solution = 01100011 [10011100].
		Maxcut = -52.97166192978911.
		Optimizer Time: 3.7796528339385986 seconds.
		Compmute Minimum Eigenvalue Time: 3.7796528339385986 seconds.


Test Case 1: 
--------------

[[1. 2. 5. 4. 3. 2. 3. 2.]
 [2. 1. 5. 4. 4. 1. 5. 2.]
 [5. 5. 5. 5. 1. 5. 4. 3.]
 [4. 4. 5. 3. 2. 5. 2. 3.]
 [3. 4. 1. 2. 1. 2. 1. 2.]
 [2. 1. 5. 5. 2. 2. 4. 4.]
 [3. 5. 4. 2. 1. 4. 5. 1.]
 [2. 2. 3. 3. 2. 4. 1. 3.]]

	Brute Force
		Solution = 00111010 [11000101].
		Maxcut = 58.0.
		Time: 0.044518029084429145 seconds.

	Variation Quantum Eignsolver (VQE):
		Solution = 11000101 [00111010].
		Maxcut = -57

In [6]:
print(f'Success: {success} / {num_tests} = {(success/num_tests) * 100} %')
print(f'Fail: {failure} / {num_tests} = {(failure/num_tests) * 100} %')

Success: 32 / 100 = 32.0 %
Fail: 68 / 100 = 68.0 %
